In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tasiilaq,65.6145,-37.6368,-4.91,96,100,2.03,GL,1675639615
1,1,castro,-24.7911,-50.0119,18.62,78,56,2.45,BR,1675639571
2,2,busselton,-33.6500,115.3333,20.07,75,26,4.64,AU,1675639789
3,3,soyo,-6.1349,12.3689,25.76,84,88,2.38,AO,1675639827
4,4,rikitea,-23.1203,-134.9692,26.17,74,3,6.78,PF,1675639557


In [12]:
city_data_df.count()

City_ID       527
City          527
Lat           527
Lng           527
Max Temp      527
Humidity      527
Cloudiness    527
Wind Speed    527
Country       523
Date          527
dtype: int64

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "StamenTerrain",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_data = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 6.5) & (city_data_df['Cloudiness'] ==0)]

# Drop any rows with null values
narrow_data = narrow_data.dropna()
# Display sample data
narrow_data.count()

City_ID       15
City          15
Lat           15
Lng           15
Max Temp      15
Humidity      15
Cloudiness    15
Wind Speed    15
Country       15
Date          15
dtype: int64

In [22]:
narrow_data.head(6)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
71,71,yulara,-25.2406,130.9889,24.00,17,0,4.81,AU,1675639645
241,241,caravelas,-17.7125,-39.2481,25.82,82,0,4.86,BR,1675639542
245,245,cayenne,4.9333,-52.3333,22.02,100,0,0.51,GF,1675639707
266,266,san patricio,28.0170,-97.5169,25.86,64,0,5.14,US,1675639591
268,268,salalah,17.0151,54.0924,22.05,73,0,1.54,OM,1675639653
272,272,dubbo,-32.2500,148.6167,25.07,47,0,3.09,AU,1675639910


In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_data[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = narrow_data.copy()
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
71,71,yulara,-25.2406,130.9889,24.00,17,0,4.81,AU,1675639645,
241,241,caravelas,-17.7125,-39.2481,25.82,82,0,4.86,BR,1675639542,
245,245,cayenne,4.9333,-52.3333,22.02,100,0,0.51,GF,1675639707,
266,266,san patricio,28.0170,-97.5169,25.86,64,0,5.14,US,1675639591,
268,268,salalah,17.0151,54.0924,22.05,73,0,1.54,OM,1675639653,
272,272,dubbo,-32.2500,148.6167,25.07,47,0,3.09,AU,1675639910,
365,365,iberia,29.9502,-91.7507,21.04,49,0,2.57,US,1675639950,
407,407,broken hill,-31.9500,141.4333,25.57,28,0,5.86,AU,1675639964,
433,433,saint-pierre,-21.3393,55.4781,25.82,78,0,2.57,RE,1675639755,
439,439,alice springs,-23.7000,133.8833,21.75,37,0,0.00,AU,1675639561,


In [25]:
# Set parameters to search for a hotel
radius = 50000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
yulara - nearest hotel: Outback Pioneer Hotel & Lodge
caravelas - nearest hotel: Pousada dos Navegantes
cayenne - nearest hotel: Central Hôtel
san patricio - nearest hotel: Best Western Plus
salalah - nearest hotel: Muscat International Hotel
dubbo - nearest hotel: Blue Gum Motel
iberia - nearest hotel: Inn of Iberia
broken hill - nearest hotel: Ibis Styles
saint-pierre - nearest hotel: Tropic Hotel
alice springs - nearest hotel: Aurora Alice Springs
lovington - nearest hotel: Hampton
san isidro - nearest hotel: Soul Mate
ormara - nearest hotel: No hotel found
buckeye - nearest hotel: Days Inn
upington - nearest hotel: Die Eiland Holiday Ressort


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
71,71,yulara,-25.2406,130.9889,24.00,17,0,4.81,AU,1675639645,Outback Pioneer Hotel & Lodge
241,241,caravelas,-17.7125,-39.2481,25.82,82,0,4.86,BR,1675639542,Pousada dos Navegantes
245,245,cayenne,4.9333,-52.3333,22.02,100,0,0.51,GF,1675639707,Central Hôtel
266,266,san patricio,28.0170,-97.5169,25.86,64,0,5.14,US,1675639591,Best Western Plus
268,268,salalah,17.0151,54.0924,22.05,73,0,1.54,OM,1675639653,Muscat International Hotel
272,272,dubbo,-32.2500,148.6167,25.07,47,0,3.09,AU,1675639910,Blue Gum Motel
365,365,iberia,29.9502,-91.7507,21.04,49,0,2.57,US,1675639950,Inn of Iberia
407,407,broken hill,-31.9500,141.4333,25.57,28,0,5.86,AU,1675639964,Ibis Styles
433,433,saint-pierre,-21.3393,55.4781,25.82,78,0,2.57,RE,1675639755,Tropic Hotel
439,439,alice springs,-23.7000,133.8833,21.75,37,0,0.00,AU,1675639561,Aurora Alice Springs


In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE